In [1]:
import numpy as np
import nibabel as nib
import os
import regex as re
import pickle
from tqdm import tqdm
import shutil

In [6]:
# string with pattern (starting with numbers)
re_number = re.compile(r'^\d+')
# string with pattern ('swar' + numbers, (can be anything in middle) ending with nii)
re_swar_number_nii = re.compile(r"^swar\d+.+\.nii$")

In [7]:
mci_func_file_paths = []
cnt = 0
main_dirs = [r"./MCI (1year)", r"./MCI (2years)", r"./MCI (3years)"]
for main_dir in main_dirs:
    for sub in tqdm(os.listdir(main_dir)):
        if not os.path.isdir(os.path.join(main_dir, sub)) or not re_number.match(sub):
            continue
        if sub.find("--") != -1:
            continue
        subject_path = os.path.join(main_dir,sub)
        for struct_func in os.listdir(subject_path):
            if not struct_func.startswith("func"):
                continue
            func_dir = os.path.join(subject_path, struct_func)
            for fmri_file in os.listdir(func_dir):
                if not re_swar_number_nii.match(fmri_file):
                    continue
                cnt += 1
                mci_func_file_paths.append(os.path.join(func_dir, fmri_file))

100%|██████████| 13/13 [00:00<00:00, 1181.96it/s]


In [2]:
def get_MCI_file_paths(modality : str) -> list[str]:
    image_paths = []
    base_path = r"./MCI_full_data/" + modality
    for year in os.listdir(base_path):
        year_path = os.path.join(base_path, year)
        for image in os.listdir(year_path):
            image_path = os.path.join(year_path, image)
            image_paths.append(image_path)
    return image_paths
    

In [3]:
mci_fmri_paths = get_MCI_file_paths('func')
mci_mri_paths = get_MCI_file_paths('struct')

mci_fmri_paths = sorted(mci_fmri_paths)
mci_mri_paths = sorted(mci_mri_paths)

In [4]:
for mri, fmri in zip(mci_fmri_paths, mci_mri_paths):
    print(mri, fmri)

./MCI_full_data/func/MCI (1year)/swar002_S_1155.nii ./MCI_full_data/struct/MCI (1year)/wm002_S_1155.nii
./MCI_full_data/func/MCI (1year)/swar003_S_0908.nii ./MCI_full_data/struct/MCI (1year)/wm003_S_0908.nii
./MCI_full_data/func/MCI (1year)/swar003_S_1122.nii ./MCI_full_data/struct/MCI (1year)/wm003_S_1122.nii
./MCI_full_data/func/MCI (1year)/swar003_S_6258.nii ./MCI_full_data/struct/MCI (1year)/wm003_S_6258.nii
./MCI_full_data/func/MCI (1year)/swar003_S_6268.nii ./MCI_full_data/struct/MCI (1year)/wm003_S_6268.nii
./MCI_full_data/func/MCI (1year)/swar003_S_6432.nii ./MCI_full_data/struct/MCI (1year)/wm003_S_6432.nii
./MCI_full_data/func/MCI (1year)/swar003_S_6479.nii ./MCI_full_data/struct/MCI (1year)/wm003_S_6479.nii
./MCI_full_data/func/MCI (1year)/swar003_S_6606.nii ./MCI_full_data/struct/MCI (1year)/wm003_S_6606.nii
./MCI_full_data/func/MCI (1year)/swar012_S_6073.nii ./MCI_full_data/struct/MCI (1year)/wm012_S_6073.nii
./MCI_full_data/func/MCI (1year)/swar012_S_6760.nii ./MCI_full_d

In [8]:
re_wm_number_nii = re.compile(r"^wm\d+.+\.nii$")

In [9]:
structuralPath = r"./MCI_structural_all"
mci_struct_file_paths = []
for years in os.listdir(structuralPath):
    year_path = os.path.join(structuralPath, years)
    if not os.path.isdir(year_path):
        continue
    for subject in os.listdir(year_path):
        subject_path = os.path.join(year_path, subject)
        if not os.path.isdir(subject_path):
            continue
        subject_mri_path = os.path.join(subject_path, "mri")
        for nii_files in os.listdir(subject_mri_path):
            if re_wm_number_nii.match(nii_files):
                mci_struct_file_paths.append(os.path.join(subject_mri_path, nii_files))

In [10]:
sorted_mci_struct_file_paths = sorted(mci_struct_file_paths)

In [12]:
unique_shapes = {}
for shape in func_shapes:
    strShape = shape.__repr__()
    if strShape not in unique_shapes:
        unique_shapes[strShape] = 0
    unique_shapes[strShape] += 1

NameError: name 'func_shapes' is not defined

In [13]:
unique_shapes

{}

In [14]:
sorted_mci_func_file_paths = sorted(mci_func_file_paths)

In [15]:
for i, j in zip(sorted_mci_func_file_paths, sorted_mci_struct_file_paths):
    # matching last path
    if i.split("/")[-1] != j.split("/")[-1]:
        print(i, j)

./MCI (1year)\002_S_1155\func\swar002_S_1155.nii ./MCI_structural_all\MCI (1year)\002_S_1155\mri\wm002_S_1155.nii
./MCI (1year)\003_S_0908\func\swar003_S_0908.nii ./MCI_structural_all\MCI (1year)\003_S_0908\mri\wm003_S_0908.nii
./MCI (1year)\003_S_1122\func\swar003_S_1122.nii ./MCI_structural_all\MCI (1year)\003_S_1122\mri\wm003_S_1122.nii
./MCI (1year)\003_S_6258\func\swar003_S_6258.nii ./MCI_structural_all\MCI (1year)\003_S_6258\mri\wm003_S_6258.nii
./MCI (1year)\003_S_6268\func\swar003_S_6268.nii ./MCI_structural_all\MCI (1year)\003_S_6268\mri\wm003_S_6268.nii
./MCI (1year)\003_S_6432\func\swar003_S_6432.nii ./MCI_structural_all\MCI (1year)\003_S_6432\mri\wm003_S_6432.nii
./MCI (1year)\003_S_6479\func\swar003_S_6479.nii ./MCI_structural_all\MCI (1year)\003_S_6479\mri\wm003_S_6479.nii
./MCI (1year)\003_S_6606\func\swar003_S_6606.nii ./MCI_structural_all\MCI (1year)\003_S_6606\mri\wm003_S_6606.nii
./MCI (1year)\012_S_6073\func\swar012_S_6073.nii ./MCI_structural_all\MCI (1year)\012_S_

In [5]:
target_shape = (79, 95, 79, 197)
original_shape = (47, 56, 47, 197)

pad_width = [(0, max(target_dim - orig_dim, 0)) for orig_dim, target_dim in zip(original_shape, target_shape)]

In [6]:
anomaly = 47
mci_func_images = []
for mci_func_path in mci_fmri_paths:
    func_image = nib.load(mci_func_path)
    if func_image.shape[0] == anomaly:
        # this particular is small, need to zero pad to make it similar to other images
        data = func_image.get_fdata()
        data = np.pad(data, pad_width=pad_width, mode='constant', constant_values=0)
        mci_func_images.append(data)
        del data
    else:
        mci_func_images.append(func_image.get_fdata())

In [7]:
mci_func_images = np.array(mci_func_images, dtype=np.float32)

In [8]:
mci_func_images.shape

(52, 79, 95, 79, 197)

In [9]:
for i in tqdm(range(len(mci_func_images))):
    j = i+1
    while j < len(mci_func_images):
        if np.all(mci_func_images[i] == mci_func_images[j]) == np.True_:
            print(i, j)
        j += 1

  0%|          | 0/52 [00:00<?, ?it/s]

100%|██████████| 52/52 [01:52<00:00,  2.17s/it]


In [17]:
np.all(mci_func_images[1] == mci_func_images[0])

np.False_

In [10]:
mci_struct_images = []
for mci_struct_path in mci_mri_paths:
    struct_image = nib.load(mci_struct_path)
    mci_struct_images.append(struct_image.get_fdata())

mci_struct_images = np.array(mci_struct_images, dtype=np.float32)

In [11]:
mci_struct_images.shape

(52, 169, 205, 169)

In [12]:
with open(r'./feature_extraction/MCI_func_52_79_95_79_197.pkl', 'wb') as f:
    pickle.dump(mci_func_images, f)

In [13]:
with open(r'./feature_extraction/MCI_struct_52_169_205_169.pkl', 'wb') as f:
    pickle.dump(mci_struct_images, f)

In [14]:
for i in sorted_mci_func_file_paths:
    print(i, end=" ")

./MCI (1year)\002_S_1155\func\swar002_S_1155.nii ./MCI (1year)\003_S_0908\func\swar003_S_0908.nii ./MCI (1year)\003_S_1122\func\swar003_S_1122.nii ./MCI (1year)\003_S_6258\func\swar003_S_6258.nii ./MCI (1year)\003_S_6268\func\swar003_S_6268.nii ./MCI (1year)\003_S_6432\func\swar003_S_6432.nii ./MCI (1year)\003_S_6479\func\swar003_S_6479.nii ./MCI (1year)\003_S_6606\func\swar003_S_6606.nii ./MCI (1year)\012_S_6073\func\swar012_S_6073.nii ./MCI (1year)\012_S_6760\func\swar012_S_6760.nii ./MCI (1year)\013_S_6206\func\swar013_S_6206.nii ./MCI (1year)\068_S_0802\func\swar068_S_0802.nii ./MCI (1year)\116_S_6428\func\swar116_S_6428.nii ./MCI (1year)\116_S_6550\func\swar116_S_6550.nii ./MCI (1year)\116_S_6775\func\swar116_S_6775.nii ./MCI (1year)\153_S_6679\func\swar153_S_6679.nii ./MCI (1year)\168_S_6180\func\swar168_S_6180.nii ./MCI (1year)\168_S_6426\func\swar168_S_6426.nii ./MCI (1year)\168_S_6467\func\swar168_S_6467.nii ./MCI (1year)\168_S_6541\func\swar168_S_6541.nii ./MCI (1year)\168_S_

In [ ]:
# copy pasting files from one path to another
paste_func_path = r"MCI_full_data\func"
paste_struct_path = r"MCI_full_data\struct"
cnt = 1
for i, j in tqdm(zip(sorted_mci_func_file_paths, sorted_mci_struct_file_paths)):
    year_count = 1
    if "1year" in i:
        year_count = 1
    elif "2years" in i:
        year_count = 2
    else:
        year_count = 3
    
    year_suffix = "s" if year_count > 1 else ""
    shutil.copyfile(i, os.path.join(paste_func_path, f"MCI ({year_count}year{year_suffix})", i.split("\\")[-1]))
    shutil.copyfile(j, os.path.join(paste_struct_path, f"MCI ({year_count}year{year_suffix})", j.split("\\")[-1]))
    print(cnt, " : done with ", i, j)
    cnt += 1

1it [00:02,  2.46s/it]

1  : done with  ./MCI (1year)\002_S_1155\func\swar002_S_1155.nii ./MCI_structural_all\MCI (1year)\002_S_1155\mri\wm002_S_1155.nii


2it [00:09,  5.03s/it]

2  : done with  ./MCI (1year)\003_S_0908\func\swar003_S_0908.nii ./MCI_structural_all\MCI (1year)\003_S_0908\mri\wm003_S_0908.nii


3it [00:16,  6.22s/it]

...

51it [05:14,  6.25s/it]

51  : done with  ./MCI (3years)\168_S_6180\func\swar168_S_6180.nii ./MCI_structural_all\MCI (3years)\168_S_6180\mri\wm168_S_6180.nii


52it [05:19,  6.15s/it]

52  : done with  ./MCI (3years)\168_S_6541\func\swar168_S_6541.nii ./MCI_structural_all\MCI (3years)\168_S_6541\mri\wm168_S_6541.nii


: 

In [12]:
len(sorted_mci_func_file_paths), len(sorted_mci_struct_file_paths)

(52, 52)

In [15]:
sorted_mci_func_file_paths[0], sorted_mci_struct_file_paths[0]

('./MCI (1year)\\002_S_1155\\func\\swar002_S_1155.nii',
 './MCI_structural_all\\MCI (1year)\\002_S_1155\\mri\\wm002_S_1155.nii')

In [24]:
CN_func_file_paths_string = r'''D:/adni datasets/CN (2years)/003_S_4119/func/swar003_S_4119.nii
D:/adni datasets/CN (2years)/003_S_6067/func/swar003_S_6067.nii
D:/adni datasets/CN (2years)/014_S_4401/func/swar014_S_4401.nii
D:/adni datasets/CN (2years)/014_S_6076/func/swar014_S_6076.nii
D:/adni datasets/CN (2years)/014_S_6145/func/swar014_S_6145.nii
D:/adni datasets/CN (2years)/014_S_6424/func/swar014_S_6424.nii
D:/adni datasets/CN (2years)/041_S_4200/func/swar041_S_4200.nii
D:/adni datasets/CN (2years)/041_S_4427/func/swar041_S_4427.nii
D:/adni datasets/CN (2years)/041_S_5100/func/swar041_S_5100.nii
D:/adni datasets/CN (2years)/041_S_6136/func/swar041_S_6136.nii
D:/adni datasets/CN (2years)/041_S_6159/func/swar041_S_6159.nii
D:/adni datasets/CN (2years)/041_S_6192/func/swar041_S_6192.nii
D:/adni datasets/CN (2years)/041_S_6226/func/swar041_S_6226.nii
D:/adni datasets/CN (2years)/041_S_6292/func/swar041_S_6292.nii
D:/adni datasets/CN (2years)/041_S_6314/func/swar041_S_6314.nii
D:/adni datasets/CN (2years)/041_S_6354/func/swar041_S_6354.nii
D:/adni datasets/CN (2years)/041_S_6401/func/swar041_S_6401.nii
D:/adni datasets/CN (2years)/041_S_6785/func/swar041_S_6785.nii
D:/adni datasets/CN (2years)/041_S_6786/func/swar041_S_6786.nii
D:/adni datasets/CN (2years)/068_S_0127/func/swar068_S_0127.nii
D:/adni datasets/CN (2years)/068_S_0473/func/swar068_S_0473.nii
D:/adni datasets/CN (2years)/068_S_4340/func/swar068_S_4340.nii
D:/adni datasets/CN (2years)/082_S_4224/func/swar082_S_4224.nii
D:/adni datasets/CN (2years)/082_S_4428/func/swar082_S_4428.nii
D:/adni datasets/CN (2years)/082_S_6283/func/swar082_S_6283.nii
D:/adni datasets/CN (2years)/082_S_6415/func/swar082_S_6415.nii
D:/adni datasets/CN (2years)/082_S_6564/func/swar082_S_6564.nii
D:/adni datasets/CN (2years)/082_S_6629/func/swar082_S_6629.nii
D:/adni datasets/CN (2years)/114_S_0416/func/swar114_S_0416.nii
D:/adni datasets/CN (2years)/114_S_6057/func/swar114_S_6057.nii'''

In [25]:
cn_func_file_paths = CN_func_file_paths_string.split("\n")

In [26]:
len(cn_func_file_paths)

30

In [27]:
cn_func_file_paths_sorted = sorted(cn_func_file_paths)

In [30]:
for file in tqdm(cn_func_file_paths_sorted):
    shutil.copyfile(file, "./CN_FUNC_DATA_SWAR_ONLY/" + file.split("/")[-1])

100%|██████████| 30/30 [02:36<00:00,  5.23s/it]


In [2]:
base_path_cn_func = r'./CN_FUNC_DATA_SWAR_ONLY/'
cn_fmri_paths = []
for image in os.listdir(base_path_cn_func):
    image_path = os.path.join(base_path_cn_func, image)
    cn_fmri_paths.append(image_path)

cn_fmri_paths = sorted(cn_fmri_paths)

In [3]:
len(cn_fmri_paths)

30

In [4]:
cn_fmri_images = []
for cn_fmri_path in tqdm(cn_fmri_paths):
    image = nib.load(cn_fmri_path)
    cn_fmri_images.append(image.get_fdata())
cn_fmri_images = np.array(cn_fmri_images, dtype=np.float32)

100%|██████████| 30/30 [00:08<00:00,  3.73it/s]


In [7]:
cn_fmri_images.shape

(30, 79, 95, 79, 197)

In [8]:
with open(r'./feature_extraction/CN_func_30_79_95_79_197.pkl', 'wb') as f:
    pickle.dump(cn_fmri_images, f)

In [2]:
cn_12_fmri_path = r'./New folder (8)/all_cn_swar_imges_12'
cn_12_mri_path = r'./New folder (8)/all_cn_wm_images_12'

def get_cn_12_file_paths(path : str) -> list[str]:
    cn_file_paths = []
    for file in os.listdir(path):
        file_path = os.path.join(path, file)
        cn_file_paths.append(file_path)
    return cn_file_paths

cn_12_fmri_paths = get_cn_12_file_paths(cn_12_fmri_path)
cn_12_mri_paths = get_cn_12_file_paths(cn_12_mri_path)

cn_12_fmri_paths = sorted(cn_12_fmri_paths)
cn_12_mri_paths = sorted(cn_12_mri_paths)

In [3]:
for i, j in zip(cn_12_mri_paths, cn_12_fmri_paths):
    print(i, j)

./New folder (8)/all_cn_wm_images_12/wm002_S_1261.nii ./New folder (8)/all_cn_swar_imges_12/swar002_S_1261.nii
./New folder (8)/all_cn_wm_images_12/wm002_S_6007.nii ./New folder (8)/all_cn_swar_imges_12/swar002_S_6007.nii
./New folder (8)/all_cn_wm_images_12/wm003_S_4288.nii ./New folder (8)/all_cn_swar_imges_12/swar003_S_4288.nii
./New folder (8)/all_cn_wm_images_12/wm003_S_4350.nii ./New folder (8)/all_cn_swar_imges_12/swar003_S_4350.nii
./New folder (8)/all_cn_wm_images_12/wm003_S_4441.nii ./New folder (8)/all_cn_swar_imges_12/swar003_S_4441.nii
./New folder (8)/all_cn_wm_images_12/wm003_S_4644.nii ./New folder (8)/all_cn_swar_imges_12/swar003_S_4644.nii
./New folder (8)/all_cn_wm_images_12/wm003_S_4900.nii ./New folder (8)/all_cn_swar_imges_12/swar003_S_4900.nii
./New folder (8)/all_cn_wm_images_12/wm003_S_6014.nii ./New folder (8)/all_cn_swar_imges_12/swar003_S_6014.nii
./New folder (8)/all_cn_wm_images_12/wm011_S_4278.nii ./New folder (8)/all_cn_swar_imges_12/swar011_S_4278.nii
.

In [15]:
cn_mri_images = []
for cn_mri_path in tqdm(cn_12_mri_paths):
    image = nib.load(cn_mri_path)
    cn_mri_images.append(image.get_fdata())
cn_mri_images = np.array(cn_mri_images, dtype=np.float32)

  0%|          | 0/12 [00:00<?, ?it/s]

100%|██████████| 12/12 [00:00<00:00, 40.26it/s]


In [16]:
with open(r'feature_extraction/cat12_CN_structural_features.pkl', 'rb') as f:
    cn_30_mri_images = pickle.load(f)
all_cn_mri_images = np.concat((cn_30_mri_images, cn_mri_images), axis=0)
all_cn_mri_images.shape

(42, 169, 205, 169)

In [17]:
with open(r'feature_extraction/CN_struct_cat_42_169_205_169.pkl', 'wb') as f:
    pickle.dump(all_cn_mri_images, f)

In [18]:
del all_cn_mri_images, cn_mri_images